# Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta parte do Projeto Prático diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você deseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

**Importação de bibliotecas**

In [1]:
import prettytable
import numpy as np
import pandas as pd
import json
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

**Importação do dataset**

In [2]:
file_path = "covtype.csv"
pd.set_option('display.max_columns', None)
             
df_cat = pd.read_csv(file_path) #Dataset com atributos categóricos
df_cat.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [3]:
url = "covtype.csv"
df = pd.read_csv(url)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [10]:
# df = df.drop(columns=['Wilderness_Area1',
#        'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
#        'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
#        'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
#        'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
#        'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
#        'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
#        'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
#        'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
#        'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
#        'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
#        'Soil_Type39', 'Soil_Type40'])
# df.head()

**1. A métrica que se deseja maximizar é o F1-Score, com média ponderada dado o desbalanceamento das classes.**

**2. O algoritmo de Machine Learning a ser treinado é a Rede Neural Artificial MLP**

In [11]:
# y = df['Cover_Type']
# X = df.drop(columns=['Cover_Type'])

# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [12]:
# X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

# X_test_std = (X_test - np.mean(X_test))/np.std(X_test)

In [13]:
# mlp = MLPClassifier(verbose=True)
# mlp.fit(X_train_std, y_train)

**Definindo parâmetros e Hiperparâmetros a serem otimizados**

In [4]:
solvers =  ['adam', 'sdg']
batch_sizes = [200, 400, 800]
learning_rate_init = [0.0001, 0.001, 0.01]
epochs = [100, 150, 200]

**3.Construindo a grade de busca**

In [6]:
models = []
for solver in solvers:
    for size in batch_sizes:
        for rate in learning_rate_init:
            for number in epochs:
                models.append([solver, size, rate, number])

print(len(models))

54


## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

**Rotina para a validação k_fold**

In [ ]:
def k_fold(folds,random)
  df_embaralhado = df.sample(frac=1, random_state=random) #Embaralha o dataset
  k=folds #Definição do número de folds
  k_folds=[] #Criação da lista de folds
  subsets=[] #Criação dos subsets
  subset_size=len(df)/k
  limite=0

  for i in range(k):  #Armazenamento de todas as linhas dentro dos subsets
    subsets.append(df.iloc[limite:limite+subset_size])
    limite=limite+subset_size

  for i in range(k):  #Armazenamento dos 5 conjuntos de Teste e Treino em k_folds
    test=subsets[i]
    train=[]
    for j in subsets:
      if j!=test:
        train.append(j)
    k_folds.append((test,train))

  return k_folds

**Validação Cruzada com melhores arquiteturas anteriores**

In [11]:
import json

# IMPORTANTE -> SÓ COMO EXEMPLO, IMPORTEI O JSON Q JÁ TINHA NO MEU NOTE, MAS A GENTE TEM Q GERAR O JSON COM AS MELHORES ARQUITETURAS
with open('resultado_melhores_arquiteturas.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

pd.DataFrame(data)

,camadas,solver,activation,iterations,media_acuracia,desvio_acuracia,media_f1,desvio_f1
0,"[48, 60]",sgd,relu,150,0.807700,0.002632,0.804039,0.002963
1,"[36, 28]",adam,logistic,150,0.798396,0.002396,0.795103,0.002576
2,"[32, 32]",sgd,relu,150,0.781922,0.003023,0.777013,0.003404
3,"[17, 8]",adam,logistic,100,0.741158,0.003078,0.732777,0.003360
4,"[12, 13]",adam,relu,100,0.741100,0.003800,0.734000,0.004000
5,"[10, 15]",adam,relu,100,0.739800,0.003800,0.732400,0.004500


In [12]:
table = prettytable.PrettyTable(['Neurônios Ocultos', 'Função de Ativação', 'Desempenho médio', 'Número de Repetições'])

for arquitetura in data:
    table.add_row([arquitetura['camadas'], 
                  arquitetura['activation'],
                  f"{arquitetura['media_f1']:.4f} +- {arquitetura['desvio_f1']:.4f}", 
                  100])

print(table)

+-------------------+--------------------+------------------+----------------------+
| Neurônios Ocultos | Função de Ativação | Desempenho médio | Número de Repetições |
+-------------------+--------------------+------------------+----------------------+
|      [48, 60]     |        relu        | 0.8040 +- 0.0030 |         100          |
|      [36, 28]     |      logistic      | 0.7951 +- 0.0026 |         100          |
|      [32, 32]     |        relu        | 0.7770 +- 0.0034 |         100          |
|      [17, 8]      |      logistic      | 0.7328 +- 0.0034 |         100          |
|      [12, 13]     |        relu        | 0.7340 +- 0.0040 |         100          |
|      [10, 15]     |        relu        | 0.7324 +- 0.0045 |         100          |
+-------------------+--------------------+------------------+----------------------+


## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html  
2. Treine o modelo com todos os dados  
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas